## node class decorator

Recalling the structure of a decision tree, it is composed of numerous nodes which is either the parent node or the leaf node. Then it is convenient to construct a object denoting the node. 

1. Attributes: 
    * in the case of binary split, the node class at least should has two attributes (denoted as self.true and self.false here) pointing to its two child nodes.
    * furthermore, it should be able to tell which child node a given data should go to. Thus information regarding the binary split should also be stored i.e. self.feature, self.dtype, self.threshold. dtype tells whether the feature involved in the split is discrete, categorical (dtype=='d') or continuous,ordinal (dtype='c').
    * additionally self.probability is the ratio of subsample size in current node to the total sample size. self.sample_index stores the index of the subsample in the total sample.
2. methods:
    * find child method returns either child node given input data.

Most importantly, if the node is leaf, it should be able to give a prediction according to the model on that leaf node. In the case of regression, this model can be e.g. the average of outcome, a linear regression, or more complex models. Thus without losing generality, a tree node decorator is defined, it takes a model class as argument and use it as a parent class to build a node class on top of it. Following is the code:

In [5]:
## define a tree node decorator: takes in a model class and create a child class out of it, then return the child class
#  to create a tree_node class e.g. node=tree_node_decorator(linear_regression)()
def tree_node_decorator(model,hparameter=None,stats=None):
    class node_model(model):
        def __init__(self,hparameter=hparameter,stats=stats):

            if not hparameter:
                super().__init__(stats=stats)              
            else:
                super().__init__(hparameter=hparameter,stats=stats)


            # tree_node: the left or true branch 
            self.true=None 

            # tree_node: the right or false branch
            self.false=None



            # parameters used in the split method: return true if the split criterion suggest going to the true child node otherwise false
            self.feature=None       
            self.dtype=None
            self.threshold=None
            
            # probability that a instance falls into current node
            self.probability=1
            
            # store the index of the subsamples belong to the current node in the total training sample
            self.sample_index=None


        def find_child(self,unit):
            '''for each unit return true or false according to the criteria'''
            if self.dtype=='c':
                return self.true if unit[self.feature] <= self.threshold else self.false

            elif self.dtype=='d':
                 return self.true if unit[self.feature] == self.threshold else self.false

            else:
                raise ValueError('The datatype of feature is incorrect. \'c\' for continuous feature \
                        and \'d\' for discrete feature')
    return node_model

Here we give a typical structure of a model that can be incorporated on the tree node:

In [ ]:
class plain_regression():
    def __init__(self,hyperparameter=0,stats=[None,None]):
    
    def online_fit(self,X,Y,feature_i):
           
    def batch_fit(self,X,Y,feature_i):            
                  
    def fit(self,X,Y):
   
    def predict(self,X):   

The key is the online_fit method. It takes input the sample data and output two list: stats_up, stats_down. The $i$th element in stats_up is a list: $[parameter,loss,status variable]$ which describesthat if the model is trained on the subsample $X[:i+1]$ what is the parameter of the trained model, the loss function after traning, and the state variables which assistants the online training or updating the other two quantities. The $i$th element in stats_down describes same quantities if trained on the subsample $X[i:]$ instead.

# tree class decorator

The tree class decorator takes in the tree_node class built from previous discussion and return a decision tree class.

**Methods** (briefly): 

* main methods: 
    The **fit** method of the decision_tree class build a tree given input data recursively by using the **build_tree** method, then the **pruning** method is applied automatically to get rid of the leaves that do not contribute to a significant amount of loss reduction. **predict** method return the corresponding prediction given data.
* other methods: 
    export_graphviz and plot_tree methods are to visualize the tree structure using the package graphviz.

**Stopping criteria**

It is important for the recursive building tree method to know when to stop. It is controlled by the two attributes: self.max_depth and self.min_sample_size. Here when the sample size is smaller than twice of the min_sample_size or the splitting results in a subsample smaller than the min_sample_size the splitting is stopped and current node is a leaf node. 

**Pruning criteria**

in the tree pruning process, whether the splitting on the parent node of two leaf nodes is kept or not depending the following formula
$$
\mathcal{L}_1 \frac{N_1}{N}+\gamma + \mathcal{L}_2 \frac{N_2}{N}+\gamma\leq (\mathcal{L}+\gamma)\lambda
$$
where again $\mathcal{L}_1$ and $\mathcal{L}_2$ are the two leaf node loss functions and $N_1$, $N_2$ their sample sizes. $\mathcal{L}$ is the parent node loss function and $N$ is the parent node sample size. $\gamma$ is a regularization parameter assign additional punishment to each split. $\lambda$ is a ratio controlling how much loss reduction is needed for the split to be significant enough. They correspond to the self.gamma and self.loss_red_thres attributes respectively. 

**optimization**

The most time consuming part of the tree algorithms is to find the threshold of splitting for ordinal feature. Since for ordinal feature, the splitting criteria is whether $X\leq thershold$, the brutal force algorithm takes $O(N^2)$ to both find the optimal threshold where $N$ is the number of values of that feature without considering the fitting time of the node model. However, if the data is sorted first, it only takes $O(N\log N)$. Thus it is optimized in time if the data is first sorted by feature. However, the data size is enlarged since we now have $d$ times the original data set with $d$ the number of features. 

This may cause a problem if the data set is too large to fit in the memory at once. However, there is a work around if the model used on the node support online training or batch training. Because now the data can be stored on hard disk and read into memory in the form of batches or sequence. At least in the case of linear regression and a model by just averaging outcome such online training or batch training method exists. The discussion can be find [here](../../Online_ML/OML_theory.ipynb)

In [6]:
## define a decision tree decorator: takes in a node class and create a child class out of it, then return the child class
#  to create a tree_node class e.g. node=tree_node_decorator(linear_regression)()

def decision_tree_decorator(tree_node,max_depth=np.Infinity,min_sample_size=2,gamma=0,loss_red_thres=0.9):
    class decision_tree():
        """
        X=np.array(shape=[samplesize,feature])
        Y=np.array(shape=[samplesize])
        feature_name_dtype=np.array([(name,'c'/'d')...])
        where 'c' respresents continuous variabl and 'd' represents discrete variable
        """
        def __init__(self,max_depth=max_depth,min_sample_size=min_sample_size,gamma=gamma):
            self.max_depth=max_depth
            self.min_sample_size=min_sample_size
            
            self.gamma=gamma
            self.loss_red_thres=loss_red_thres
                                 


            # quantities used in building tree
            self.root=None
            self.feature_name_dtype=None

        def fit(self,X=[],Y=[],XY_mul=[],feature_name_dtype=None):
            '''
            feature_name_dtype=[feature_name,'c' for ordinal feature and 'd' for categorical feature]
            hparameter: hyperparameters for the model of on the tree node 
            '''
            self.feature_name_dtype=np.array(feature_name_dtype)
            
            if len(X)!=0 and len(Y)!=0:
        
                # just to store the root of the tree 
                # consistent check whether the shape of inputs are expected:
                condition1= ((len(np.shape(X))==2)
                            and (len(np.shape(self.feature_name_dtype))==2))         
                condition2= ((np.shape(X)[0]==np.shape(Y)[0]) 
                             and (np.shape(X)[1]==np.shape(self.feature_name_dtype)[0]))

                if condition1 and condition2:
                    if len(np.shape(Y))==1: 
                        Y = np.expand_dims(Y, axis=1)

                    # Add Y,g,h as last columns to X so easier to split them together
                    XY = np.concatenate((X, Y), axis=1)

                else:
                    raise ValueError('The shapes of inputs are not compatible')

                # store data 
                self.XY_mul=np.array([sorted(XY,key=lambda unit: unit[i]) for i in range(X.shape[1])])                               
                
            elif len(XY_mul)!=0:
                self.XY_mul=XY_mul
                
            else:
                raise ValueError('No input data is given')
                
            currnode_index=np.ones(self.XY_mul.shape[:-1],dtype=bool)
                                       
            self.root=self.build_tree(currnode_index=currnode_index)

            # pruning trees
            self.pruning(self.root)

            # release memory
            self.XY_mul=0


        def build_tree(self,currnode_index=True,depth=1,stats=None,probability=1):   
            
            
            # pick the units out of the total samples that correspond to current node determined by the currnode_index
            XY_mul=self.XY_mul[currnode_index].reshape(len(self.feature_name_dtype),-1,len(self.feature_name_dtype)+1)
            
            parent_condition=((depth<self.max_depth) and (XY_mul.shape[1]>=2*self.min_sample_size))
            
            leaf_condition=((depth==self.max_depth) or (XY_mul.shape[1]<2*self.min_sample_size))
            
            null_condtion=((depth>self.max_depth) or (XY_mul.shape[1]<self.min_sample_size))
            
            if null_condtion:
                return None
            
            elif leaf_condition:
                node=tree_node(stats=stats)
                node.fit(XY_mul[0,:,:-1],XY_mul[0,:,-1:])             
                node.probability=probability
                node.sample_index=currnode_index
                
                return node
                    
            elif parent_condition:    
                # this node has the potential to be parent
                node=tree_node(stats=stats)                    
                node.probability=probability
                node.sample_index=currnode_index

                best=[np.Infinity,0,0]  ##[best split loss, feature_i whic gives best split, value of feature_i gives best split]
                best_split_stats=[[],[]]     ##[left split stats,right split stats]
                best_split_left_size=0
                best_split_right_size=0

                for feature_i in range(len(XY_mul)):
                    if self.feature_name_dtype[feature_i][1]=='c':
                        b_split_left_stats,b_split_right_stats,b_split_left_size,b_split_right_size,b_split_value,b_loss\
                        =node.online_fit(XY_mul[feature_i,:,:-1],XY_mul[feature_i,:,-1:],feature_i,feature_name_dtype=self.feature_name_dtype)
                    elif self.feature_name_dtype[feature_i][1]=='d':
                        b_split_left_stats,b_split_right_stats,b_split_left_size,b_split_right_size,b_split_value,b_loss\
                        =node.batch_fit(XY_mul[feature_i,:,:-1],XY_mul[feature_i,:,-1:],feature_i,feature_name_dtype=self.feature_name_dtype)

                    if b_loss<best[0]:
                        best[0]=b_loss
                        best[1]=feature_i
                        best[2]=b_split_value

                        best_split_stats[0]=b_split_left_stats
                        best_split_stats[1]=b_split_right_stats
                        
                        best_split_left_size=b_split_left_size
                        best_split_right_size=b_split_right_size
                        
                # whether disorder reduction is significant enough to support the division
                condition=(best_split_left_size>=self.min_sample_size) and (best_split_right_size>=self.min_sample_size)
                
                if condition: 
                # this is truly a parent node
                    if self.feature_name_dtype[best[1],1]=='c':
                        node.dtype='c'        
                        left_index=(self.XY_mul[:,:,best[1]]<=best[2]) & (currnode_index)
                        right_index=(self.XY_mul[:,:,best[1]]>best[2]) & (currnode_index)
                    else:
                        node.dtype='d'
                        left_index=(self.XY_mul[:,:,best[1]]==best[2]) & (currnode_index)
                        right_index=(self.XY_mul[:,:,best[1]]!=best[2]) & (currnode_index)

                    node.feature=best[1]
                    node.threshold=best[2]
                    
                    
                    pro_left=best_split_left_size/(best_split_left_size+best_split_right_size)*node.probability
                    pro_right=best_split_right_size/(best_split_left_size+best_split_right_size)*node.probability
                    
                    ## add child branches
                    node.true=self.build_tree(currnode_index=left_index,depth=depth+1,stats=best_split_stats[0],probability=pro_left)
                    node.false=self.build_tree(currnode_index=right_index,depth=depth+1,stats=best_split_stats[1],probability=pro_right)                       
                
                return node 
            
        def pruning(self,currnode):
            if currnode.true is None:
                return currnode
            else:
                # prune child nodes first
                currnode.true=self.pruning(currnode.true)
                currnode.false=self.pruning(currnode.false)
                
                # whether current node is the parent of two leaf nodes
                if currnode.true.true is None and currnode.false.false is None:
                    loss_leaves=(currnode.true.loss*currnode.true.probability\
                                +currnode.false.loss*currnode.false.probability+2*self.gamma)
                    loss_parent=currnode.loss*currnode.probability+self.gamma
                    
                    # whether loss reduction is too small: need to be removed
                    if (loss_parent!=0) and (loss_leaves/loss_parent>=self.loss_red_thres):
                        currnode.true=None
                        currnode.false=None
                return currnode
            
        def export_graphviz(self):
            # BFS trasverse the tree structure         
            def get_name(currnode):
                if not (currnode.true is None):                
                    if currnode.dtype=='c':
                        curr_name='{}<={}'.format(self.feature_name_dtype[currnode.feature][0],currnode.threshold)
                    else:
                        curr_name='{} is {}'.format(self.feature_name_dtype[currnode.feature][0],currnode.threshold)
                else:
                    curr_name=currnode.model_description()
                return curr_name
            
            dot = Digraph(comment='Decision Tree')
            que=[self.root]
            currnode=que[0]
            parent_name=get_name(currnode)
            dot.node(parent_name)
            
            while que:
                currnode=que[0]
                parent_name=get_name(currnode)
                if not (que[0].true is None):
                    currnode=que[0].true
                    true_child_name=get_name(currnode)
                    dot.node(true_child_name)
                    dot.edge(parent_name,true_child_name)
                    que.append(currnode)
                    
                    currnode=que[0].false
                    false_child_name=get_name(currnode)                  
                    dot.node(false_child_name)
                    dot.edge(parent_name,false_child_name)
                    que.append(currnode)                            
         
                que.pop(0)
            
            return dot
                                   
        def plot_tree(self):
            self.export_graphviz().render(view=True)
            


        def predict(self,X):
            if self.root==None:
                raise Exception('model not fitted yet')
            else:
                predictions=np.array([])
                if len(np.shape(X))==2 and np.shape(X)[1]==len(self.feature_name_dtype):
                    for instance in X:
                        currnode=self.root
                        while currnode.true!=None: # the current node has branches
                            currnode=currnode.find_child(instance)
                        predictions=np.concatenate((predictions,currnode.predict(instance)),axis=0)  
                    return predictions
                else:
                    raise ValueError('the shape of X should be (sample,feature)')
         
            
    return decision_tree

In [8]:
class average_regression():
    def __init__(self,hparameter=0,stats=None):
        self.lambd=hparameter
        if stats is None:
            self.parameter=0     
            self.loss=0
        else:
            self.parameter=stats[0]      
            self.loss=stats[1]
            


    
    def online_fit(self,X,Y,feature_i,feature_name_dtype=None):
        '''calculate stats for every binary division of the XY into XY[:i] and XY[i:] with i in range(len(XY))
           stats_up: store information about subsample XY[:i]
           stats_down: store information about subsample XY[i:]
        '''
        ## quantites record information about the best split
        N=len(Y)
        
        b_split_value=None    
        b_loss=np.Infinity  
        b_split_left_size=1
        b_split_right_size=N-1
        
        b_split_left_stats=None
        b_split_right_stats=None
               
        
        
        # initialize stats_up
        stats_up=[[] for i in range(len(Y))] 
        stats_up[0].append(self.partial_fit(X[0:1],Y[0:1]))                       # value of omega
        stats_up[0].append(self.loss_cal(X[0:1],Y[0:1]))                          # value of loss

       
        # initialize stats_down
        if not self.parameter:
            self.fit(X,Y)
        if not self.loss:
            self.loss=self.loss_cal(X,Y)

        stats_down=[[] for i in range(len(Y))]
        stats_down[0].append(self.parameter)
        stats_down[0].append(self.loss)
        
        
        for i in range(1,len(Y)):
            
            # update stats_up
            omega_i=stats_up[i-1][0]*i/(i+1)+Y[i,0]/(1+self.lambd)/(i+1)
            loss_i=((stats_up[i-1][1]+(1+self.lambd)*stats_up[i-1][0]**2)*i/(i+1)+Y[i,0]**2/(i+1))\
                   -(1+self.lambd)*(omega_i**2)
            stats_up[i].append(omega_i)
            stats_up[i].append(loss_i)


            # update stats_down
            omega_i_1=stats_down[i-1][0]*(N-i+1)/(N-i)-Y[i-1,0]/(1+self.lambd)/(N-i)
            loss_i_1=((stats_down[i-1][1]+(1+self.lambd)*stats_down[i-1][0]**2)*(N-i+1)/(N-i)-Y[i-1,0]**2/(N-i))\
                   -(1+self.lambd)*(omega_i_1**2)
            stats_down[i].append(omega_i_1)
            stats_down[i].append(loss_i_1)
            
            # find out the best split point
            if X[i,feature_i]!=X[i-1,feature_i] or i==len(Y):
                if stats_up[i-1][-1]*i/N+stats_down[i][-1]*(N-i)/N<b_loss:
                    b_split_left_stats=stats_up[i-1]
                    b_split_right_stats=stats_down[i]
                    b_split_left_size=i
                    b_split_right_size=N-i
                    
                    b_loss=stats_up[i-1][-1]*i/N+stats_down[i][-1]*(N-i)/N
                    b_split_value=X[i-1,feature_i]
                   
        return b_split_left_stats,b_split_right_stats,b_split_left_size,b_split_right_size,b_split_value,b_loss
    
    def batch_fit(self,X,Y,feature_i,feature_name_dtype=None):
        '''calculate stats for every binary division of the XY into XY[:,feature_i]==value and XY[:,feature_i]!=value 
           for all possible values of feature_i
           omega_batch,loss_batch: information about subsample XY[:,feature_i]==value
           omega_ba_null,loss_ba_null: information about subsample XY[:,feature_i]!=value 
        '''
        N=len(Y)
        
        b_split_left_stats=None
        b_split_right_stats=None
        b_split_left_size=1
        b_split_right_size=N-1
        
        best_split_value=None
        best_loss=np.Infinity
        
        # initialize stats_down
        if not self.parameter:
            self.fit(X,Y)
        if not self.loss:
            self.loss=self.loss_cal(X,Y)
            
        omega_tot=self.parameter
        loss_tot=self.loss
        
        
        
        #l,r: left bound and right bound of the batch
        l=0
        r=1
        while r<len(Y):
            while r<len(X) and X[r,feature_i]==X[r-1,feature_i]:
                r+=1
                
            omega_batch=self.partial_fit(X[l:r],Y[l:r])
            loss_batch=self.loss_cal(X[l:r],Y[l:r])
                       
            omega_ba_null=omega_tot*N/(N-(r-l))-np.sum(Y[l:r])/(1+self.lambd)/(N-(r-l))
            loss_ba_null=((loss_tot+(1+self.lambd)*omega_tot**2)*N/(N-(r-l))-np.sum(Y[l:r]**2)/(N-(r-l)))\
                   -(1+self.lambd)*(omega_ba_null**2)
            
            
            if loss_batch*(r-l)/N+loss_ba_null*(N-(r-l))/N<best_loss:
                b_split_left_stats=[omega_batch,loss_batch]
                b_split_right_stats=[omega_ba_null,loss_ba_null]
                b_split_left_size=r-l
                b_split_right_size=N-(r-l)
                
                best_loss=loss_batch*(r-l)/N+loss_ba_null*(N-(r-l))/N
                best_split_value=X[r-1,feature_i]               
            
            l=r
            r+=1
            
        return b_split_left_stats,b_split_right_stats,b_split_left_size,b_split_right_size,best_split_value,best_loss
            
                  
    def partial_fit(self,X,Y,feature_name_dtype=None):
        # conventional stochastic gradient method to train model
        return np.mean(Y)/(1+self.lambd)
    
    def fit(self,X,Y,feature_name_dtype=None):
        # conventional stochastic gradient method to train model
        self.parameter=np.mean(Y)/(1+self.lambd)
    
    def predict(self,X,feature_name_dtype=None):
        if len(np.shape(X))==2:
            return np.array([self.parameter for i in range(X.shape[0])])        
        elif len(np.shape(X))==1:
            return np.array([self.parameter])
        else:
            raise ValueError('The shape of input data is incorrect')
    
    def loss_cal(self,X,Y,feature_name_dtype=None):
        return np.mean(Y**2)-np.mean(Y)**2/(1+self.lambd)
    
    def model_description(self):
        return 'predicted value is {}'.format(self.parameter)

In [11]:
class linear_regression():
    def __init__(self,hparameter=None,stats=None):
        if hparameter is None:
            self.lambd=0
            self.sigma=0.05
        else:
            self.lambd=hparameter[0]
            self.sigma=hparameter[1]

        if stats is None:
            self.Gamma=None
            self.theta=None    
            self.loss=None
        else:           
            self.Gamma=stats[0]
            self.theta=stats[1]      
            self.loss=stats[2]


    
    def online_fit(self,X,Y,feature_i,feature_name_dtype=None):
        '''calculate stats for every binary division of the XY into XY[:i] and XY[i:] with i in range(len(XY))
           stats_up: store information about subsample XY[:i]
           stats_down: store information about subsample XY[i:]
        '''
        X_feature_i=X[:,feature_i]
     
        if not (feature_name_dtype is None):
            X=X[:,feature_name_dtype[:,1]=='c']

        N=len(Y)  
        d=X.shape[1]+1
        
        b_split_value=None    
        b_loss=np.Infinity  
        b_split_left_size=1
        b_split_right_size=N-1
        
        b_split_left_stats=None
        b_split_right_stats=None
               
        
        
        # initialize stats_up
        stats_up=[[] for i in range(len(Y))] 
        Gamma,theta_hat,M=self.partial_fit(X[:d],Y[:d])
        # initialize the first d elements by the 
        for i in range(d):
            stats_up[i].append(Gamma)
            stats_up[i].append(theta_hat)
            stats_up[i].append(M)                        # value of loss

       
        # initialize stats_down
        if self.theta is None:
            self.fit(X[d:],Y[d:])
            
        stats_down=[[] for i in range(len(Y))]
        for i in range(d):
            stats_down[i].append(self.Gamma)
            stats_down[i].append(self.theta)
            stats_down[i].append(self.loss)

        
        X=np.concatenate((X,np.ones((X.shape[0],1))),axis=1)
        
        for i in range(d,X.shape[0]-d):         
        #updata stats_up
            # update Gamma matrix

            half=np.tensordot(stats_up[i-1][0],X[i],axes=[1,0]) 
            numerator=np.tensordot(np.expand_dims(half,1),np.expand_dims(half,1),axes=[1,1])
            
            denominator=(1+np.tensordot(X[i],half,axes=[0,0]))          
            Gammai=stats_up[i-1][0]-numerator/denominator
            stats_up[i].append(Gammai)

            misspred=np.tensordot(stats_up[i-1][1],X[i],axes=[0,0])-Y[i]

            # update theta_hat   
            theta_hati=stats_up[i-1][1]-np.tensordot(stats_up[i][0],X[i],axes=[1,0])*misspred
            stats_up[i].append(theta_hati)


            # update mean square error
            Mi=i/(i+1)*stats_up[i-1][2]+misspred*misspred/(denominator*(i+1))    
            stats_up[i].append(Mi)
            
         #updata stats_down   
            # update Gamma matrix
            half=np.tensordot(stats_down[i-1][0],X[i-1],axes=[1,0]) 
            numerator=np.tensordot(np.expand_dims(half,1),np.expand_dims(half,1),axes=[1,1])
            denominator=(1-np.tensordot(X[i-1],half,axes=[0,0]))
            Gammai=stats_down[i-1][0]+numerator/denominator
            stats_down[i].append(Gammai)

            misspred=np.tensordot(stats_down[i-1][1],X[i-1],axes=[0,0])-Y[i-1]

            # update theta_hat   
            theta_hati=stats_down[i-1][1]+np.tensordot(stats_down[i][0],X[i-1],axes=[1,0])*misspred
            stats_down[i].append(theta_hati)


            # update mean square error
            Mi=(N-(i-1))/(N-i)*stats_down[i-1][2]-misspred*misspred/(denominator*(N-i))    
            stats_down[i].append(Mi)
            
        # find out the best split point
            if  X_feature_i[i]!=X_feature_i[i-1] or i==len(Y):
                if stats_up[i-1][-1]*i/N+stats_down[i][-1]*(N-i)/N<b_loss:
                    b_split_left_stats=stats_up[i-1]
                    b_split_right_stats=stats_down[i]
                    b_split_left_size=i
                    b_split_right_size=N-i
                    
                    b_loss=stats_up[i-1][-1]*i/N+stats_down[i][-1]*(N-i)/N
                    b_split_value=X_feature_i[i-1]
                   
        return b_split_left_stats,b_split_right_stats,b_split_left_size,b_split_right_size,b_split_value,b_loss
        #return stats_up,stats_down
    
    def batch_fit(self,X,Y,feature_i,feature_name_dtype=None):
        '''calculate stats for every binary division of the XY into XY[:,feature_i]==value and XY[:,feature_i]!=value 
           for all possible values of feature_i
           omega_batch,loss_batch: information about subsample XY[:,feature_i]==value
           omega_ba_null,loss_ba_null: information about subsample XY[:,feature_i]!=value 
        '''
        X_feature_i=X[:,feature_i]
        
        if not (feature_name_dtype is None):
            X=X[:,feature_name_dtype[:,1]=='c']                 
       
        N=len(Y)
               
        d=X.shape[1]+1
        
        b_split_left_stats=None
        b_split_right_stats=None
        b_split_left_size=1
        b_split_right_size=N-1
        
        best_split_value=None
        best_loss=np.Infinity
        
        # initialize stats_down

        if self.theta is None:
            self.fit(X[d:],Y[d:])

        
        X=np.concatenate((X,np.ones((X.shape[0],1))),axis=1)

        stats_batch=[]
        stats_ba_null=[]
        
        #l,r: left bound and right bound of the batch
        l=0
        r=1
        while r<len(Y):
            while r<len(X) and X_feature_i[r]==X_feature_i[r-1]:
                r+=1
                
            stats_batch=self.partial_fit(X[l:r],Y[l:r])
            stats_ba_null=self.partial_fit(np.concatenate(X[:l],X[r:],axis=0),np.concatenate(Y[:l],Y[r:],axis=0))
                   
            if stats_batch[-1]*(r-l)/N+stats_ba_null[-1]*(N-(r-l))/N<best_loss:
                b_split_left_stats=stats_batch
                b_split_right_stats=stats_ba_null
                b_split_left_size=r-l
                b_split_right_size=N-(r-l)
                
                best_loss=stats_batch[-1]*(r-l)/N+stats_ba_null[-1]*(N-(r-l))/N
                best_split_value=X_feature_i[r-1]               
            
            l=r
            r+=1
            
        return b_split_left_stats,b_split_right_stats,b_split_left_size,b_split_right_size,best_split_value,best_loss
            
                  
    def partial_fit(self,X,Y,feature_name_dtype=None): 
        if not (feature_name_dtype is None):
            X=np.concatenate((X[:,feature_name_dtype[:,1]=='c'],np.ones((X.shape[0],1))),axis=1)
        else:
            X=np.concatenate((X,np.ones((X.shape[0],1))),axis=1)
            
        N=len(Y)    
        a=np.zeros([X.shape[1],X.shape[1]])
        b=np.zeros([X.shape[1]])
        c=np.zeros([1])
        for i in range(N):
            a+=np.tensordot(np.expand_dims(X[i],1),np.expand_dims(X[i],1),axes=[1,1])
            b+=X[i]*Y[i]
            c+=Y[i]*Y[i]

        Gamma=np.linalg.inv(a+self.sigma*np.identity(X.shape[1]))
        # use normal equation to calculate theta
        theta_hat=np.tensordot(Gamma,b,axes=[1,0])
        # use normal equation to calculate Mean Square Error
        M=(-np.tensordot(np.tensordot(b,Gamma,axes=[0,0]),b,axes=[0,0])+c)/N
 
        return [Gamma,theta_hat,M]
    
    def fit(self,X,Y,feature_name_dtype=None):    
        if not (feature_name_dtype is None):
            X=np.concatenate((X[:,feature_name_dtype[:,1]=='c'],np.ones((X.shape[0],1))),axis=1)
        else:
            X=np.concatenate((X,np.ones((X.shape[0],1))),axis=1)
            
        a=np.zeros([X.shape[1],X.shape[1]])
        b=np.zeros([X.shape[1]])
        c=np.zeros([1])
        for i in range(X.shape[0]):
            a+=np.tensordot(np.expand_dims(X[i],1),np.expand_dims(X[i],1),axes=[1,1])
            b+=X[i]*Y[i]
            c+=Y[i]*Y[i]

        Gamma=np.linalg.inv(a+self.sigma*np.identity(X.shape[1]))
        # use normal equation to calculate theta
        theta_hat=np.tensordot(Gamma,b,axes=[1,0])
        # use normal equation to calculate Mean Square Error
        M=(-np.tensordot(np.tensordot(b,Gamma,axes=[0,0]),b,axes=[0,0])+c)/X.shape[0]
        
        self.Gamma=Gamma
        self.theta=theta_hat
        self.loss=M

    
    
    def predict(self,X,feature_name_dtype=None):
          
        if len(np.shape(X))==2:
            if not (feature_name_dtype is None):
                X=np.concatenate((X[:,feature_name_dtype[:,1]=='c'],np.ones((X.shape[0],1))),axis=1)
            else:
                X=np.concatenate((X,np.ones((X.shape[0],1))),axis=1)

            return np.tensordot(self.theta,X,axes=[0,1])   
        
        elif len(np.shape(X))==1:
            if not (feature_name_dtype is None):
                X=np.concatenate((X[feature_name_dtype[:,1]=='c'],[1]),axis=0)
            else:
                X=np.concatenate((X,[1]),axis=0)               
            return np.array([np.tensordot(self.theta,X,axes=[0,0])])
        else:
            raise ValueError('The shape of input data is incorrect')
    def model_description(self):
        model_eq='y='
        for i in range(len(self.theta)-1):
            model_eq+='+{}*X_{}'.format(round(self.theta[i],5),i) if self.theta[i]>=0 \
                      else '{}*X_{}'.format(round(self.theta[i]),i)
        model_eq+='{}'.format(self.theta[-1])
        return model_eq

In [13]:
def xgboost_tree_decorator(decision_tree,max_num_trees=10):
    class xgboost_tree():
        """
        X=np.array(shape=[samplesize,feature])
        Y=np.array(shape=[samplesize])
        feature_name_type=np.array([(name,'c'/'d')...])
        where 'c' respresents continuous variabl and 'd' represents discrete variable
        """
        def __init__(self,max_num_trees=max_num_trees):
            self.max_num_trees=max_num_trees          


        def fit(self,X,Y,feature_name_dtype):

            # just to store the root of the tree 
            # consistent check whether the shape of inputs are expected:
            condition1= ((len(np.shape(X))==2)
                        and (len(np.shape(feature_name_dtype))==2))         
            condition2= ((np.shape(X)[0]==np.shape(Y)[0]) 
                         and (np.shape(X)[1]==np.shape(feature_name_dtype)[0]))

            if condition1 and condition2:
                if len(np.shape(Y))==1: 
                    Y = np.expand_dims(Y, axis=1)

                # Add Y,g,h as last columns to X so easier to split them together
                XY = np.concatenate((X, Y), axis=1)

            else:
                raise ValueError('The shapes of inputs are not compatible')

            # store data 
            self.XY_mul=np.array([sorted(XY,key=lambda unit: unit[i]) for i in range(X.shape[1])])
            self.trees=[]
            self.feature_name_dtype=feature_name_dtype
            
        
                  
            for i in range(self.max_num_trees):
                tree_i=decision_tree()
                tree_i.gamma=tree_i.gamma/(i+1)
                tree_i.fit(XY_mul=self.XY_mul,feature_name_dtype=feature_name_dtype)
                self.update_Y(tree_i.root)
                self.trees.append(tree_i)
                                        
            # release memory
            self.XY_mul=0


        def predict(self,X):
            result=np.zeros((len(X)))
            for tree_i in self.trees:
                result+=tree_i.predict(X)
                
            return result 
        
        def plot_tree(self):
            dot = Digraph(comment='XGboost Trees')
            for tree_i in self.trees:
                dot.subgraph(tree_i.export_graphviz())
            dot.render(view=True)
        
        def update_Y(self,node):
            if node.true is None:
                y_hat=node.predict(self.XY_mul[node.sample_index][:,:-1])     
                y_hat=np.expand_dims(y_hat,axis=1)
                zeros=np.zeros((len(y_hat),len(self.feature_name_dtype)))
                y_hat=np.concatenate((zeros,y_hat),axis=1)      
                self.XY_mul[node.sample_index]-=y_hat
                node.sampel_index=None
            else:
                node.sampel_index=None
                self.update(node.true)
                self.update(node.false)
                      
    return xgboost_tree